In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import sys
import os

sys.path.append(os.path.abspath('../'))

del sys, os

In [3]:
import pandas as pd
import numpy as np

from torch import nn

from time import time

In [4]:
import matplotlib.pyplot as plt

In [5]:
# LaTeX rendering in graphs
from distutils.spawn import find_executable

if find_executable('latex'):
    plt.rc('text', usetex=True)
else:
    print('LaTeX not found...')

plt.rc('font', family='serif')

# High resolution graphs
%config InlineBackend.figure_format = 'retina'

# VAE

### Model Training

We ran the training of our model on GPU, using the specified hyperparameters and architecture. We obtain a validation ELBO of -95.90, which corresponds to the desired objective. 

### Evaluating log-likelihood with Importance Samping